# Mistral Large 3: Frontier Open-Source Multimodal Model on Amazon Bedrock

---

[Mistral Large 3](https://mistral.ai/news/mistral-3) represents a significant leap forward in open-source AI, introducing a powerful sparse Mixture-of-Experts (MoE) architecture with **41 billion active parameters** out of **675 billion total parameters**. Released under the Apache 2.0 license, this model delivers frontier-level performance while remaining fully open source.

## Key Highlights

- **Multimodal Capabilities**: Native image understanding enables vision-language tasks
- **Multilingual Excellence**: Best-in-class performance across 40+ languages
- **Efficient Architecture**: Sparse MoE design provides excellent performance-to-cost ratio
- **Advanced Reasoning**: State-of-the-art conversational and reasoning capabilities
- **Open Source**: Apache 2.0 license for commercial and non-commercial use

In this notebook, we demonstrate Mistral Large 3's capabilities using Amazon Bedrock, covering:

1. Basic text generation and reasoning
2. Multilingual capabilities
3. Vision and multimodal understanding
4. Complex reasoning tasks

---

## Model Card

---

| Specification | Details |
|--------------|----------|
| **Model ID** | `mistral.mistral-large-3-675b-instruct` |
| **Architecture** | Sparse Mixture-of-Experts (MoE) |
| **Active Parameters** | 41B |
| **Total Parameters** | 675B |
| **License** | Apache 2.0 |
| **Multimodal** | Yes (Text + Vision) |
| **Languages** | 40+ languages supported |
| **Training Infrastructure** | 3000 NVIDIA H200 GPUs |

### Performance

- Ranks **#2** among open-source non-reasoning models
- Achieves parity with best instruction-tuned open-weight models
- Best-in-class multilingual conversation performance

---

## Getting Started

---

In [ ]:
# Install required dependencies
%pip install --upgrade --quiet boto3 botocore

In [ ]:
import boto3
import json
import base64
from botocore.exceptions import ClientError

In [ ]:
# Configuration
MODEL_ID = "mistral.mistral-large-3-675b-instruct"
REGION = "us-west-2"  # Update based on model availability

# Initialize Bedrock client
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=REGION
)

print(f"Model ID: {MODEL_ID}")
print(f"Region: {REGION}")

In [ ]:
def converse(messages, system_prompt=None, max_tokens=5000, temperature=0.1, tool_config=None):
    """
    Helper function to interact with Mistral Large 3 using the Converse API.
    
    Args:
        messages: List of message dictionaries with 'role' and 'content'
        system_prompt: Optional system prompt string
        max_tokens: Maximum tokens to generate
        temperature: Sampling temperature (0.0 to 1.0)
        tool_config: Optional tool configuration for function calling
    
    Returns:
        Response from the model
    """
    converse_params = {
        "modelId": MODEL_ID,
        "messages": messages,
        "inferenceConfig": {
            "maxTokens": max_tokens,
            "temperature": temperature
        }
    }
    
    if system_prompt:
        converse_params["system"] = [{"text": system_prompt}]
    
    if tool_config:
        converse_params["toolConfig"] = tool_config
    
    response = bedrock_client.converse(**converse_params)
    return response

In [ ]:
def get_response_text(response):
    """Extract text content from a Converse API response."""
    return response['output']['message']['content'][0]['text']

---

## 1. Basic Text Generation

Let's start with basic text generation to demonstrate Mistral Large 3's capabilities.

---

In [ ]:
# Simple question answering
messages = [
    {
        "role": "user",
        "content": [{"text": "What are the key differences between supervised and unsupervised machine learning? Provide a concise explanation with examples."}]
    }
]

response = converse(messages, temperature=0.3)
print(get_response_text(response))

### Code Generation

Mistral Large 3 excels at code generation tasks.

In [ ]:
code_prompt = """
Write a Python function that implements a binary search algorithm. 
Include:
1. Type hints
2. Docstring with examples
3. Handle edge cases
"""

messages = [
    {"role": "user", "content": [{"text": code_prompt}]}
]

response = converse(messages, temperature=0.1)
print(get_response_text(response))

---

## 2. Multilingual Capabilities

Mistral Large 3 supports 40+ languages with best-in-class multilingual conversation performance.

---

In [ ]:
# Translation demonstration
translation_prompt = """
Translate the following English text into French, German, Japanese, and Spanish:

"Artificial intelligence is transforming how we interact with technology, 
enabling more natural and intuitive experiences across all devices."

Format each translation with the language name followed by the translation.
"""

messages = [
    {"role": "user", "content": [{"text": translation_prompt}]}
]

response = converse(messages, temperature=0.3)
print(get_response_text(response))

In [ ]:
# Multilingual conversation - responding in the same language as the query
multilingual_queries = [
    ("French", "Expliquez-moi le concept de l'apprentissage automatique en termes simples."),
    ("Japanese", "クラウドコンピューティングの利点を簡単に説明してください。"),
    ("Spanish", "¿Cuáles son las mejores prácticas para la seguridad en la nube?")
]

for language, query in multilingual_queries:
    print(f"\n{'='*60}")
    print(f"Language: {language}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    messages = [{"role": "user", "content": [{"text": query}]}]
    response = converse(messages, temperature=0.5)
    print(f"\nResponse:\n{get_response_text(response)}")

### Cross-lingual Analysis

Analyze content in one language and respond in another.

In [ ]:
cross_lingual_prompt = """
Analyze the following German text and provide a summary in English:

"Die künstliche Intelligenz hat in den letzten Jahren enorme Fortschritte gemacht. 
Besonders im Bereich der Sprachverarbeitung und Bildererkennung wurden bedeutende 
Durchbrüche erzielt. Diese Technologien finden zunehmend Anwendung in verschiedenen 
Branchen, von der Medizin bis zur Automobilindustrie."

Provide:
1. English summary
2. Key topics mentioned
3. Sentiment analysis
"""

messages = [{"role": "user", "content": [{"text": cross_lingual_prompt}]}]
response = converse(messages, temperature=0.3)
print(get_response_text(response))

---

## 3. Vision and Multimodal Understanding

Mistral Large 3 features native image understanding capabilities. 

> **Note**: For vision tasks, use the **InvokeModel API** with Mistral's native message format. Converse API support for images is coming soon.

---

In [ ]:
import urllib.request

def load_image_as_base64(image_path_or_url):
    """
    Load an image from a file path or URL and convert to base64.
    
    Args:
        image_path_or_url: Local file path or URL to the image
    
    Returns:
        Base64 encoded string of the image
    """
    if image_path_or_url.startswith(('http://', 'https://')):
        req = urllib.request.Request(
            image_path_or_url,
            headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        )
        with urllib.request.urlopen(req) as response:
            image_data = response.read()
    else:
        with open(image_path_or_url, 'rb') as f:
            image_data = f.read()
    
    return base64.standard_b64encode(image_data).decode('utf-8')

In [ ]:
def analyze_image(image_source, prompt, media_type="image/jpeg"):
    """
    Analyze an image using Mistral Large 3's vision capabilities.
    
    Note: Vision requires using the InvokeModel API with Mistral's native format.
    
    Args:
        image_source: File path or URL to the image
        prompt: Text prompt describing what to analyze
        media_type: MIME type of the image (default: image/jpeg)
    
    Returns:
        Model's analysis of the image
    """
    image_base64 = load_image_as_base64(image_source)
    
    # Mistral's native multimodal format using image_url with base64 data
    payload = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:{media_type};base64,{image_base64}"
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "max_tokens": 1024,
        "temperature": 0.3
    }
    
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(payload)
    )
    
    result = json.loads(response['body'].read())
    return result['choices'][0]['message']['content']

In [ ]:
# Example: Analyze a financial chart
image_path = "../Pixtral-samples/Pixtral_data/Amazon_Chart.png"

analysis_prompt = """Analyze this chart and provide:
1. What type of data is being shown
2. Key metrics and their values
3. Notable trends or patterns
"""

result = analyze_image(image_path, analysis_prompt, media_type="image/png")
print(result)

### Additional Image Analysis

Analyze different types of images including photos, diagrams, and more.

In [ ]:
# Example: Analyze a data table image
image_path = "../Pixtral-samples/Pixtral_data/Crosstab_of_Cola_Preference_by_Age_and_Gender.png"

table_prompt = """Analyze this table and provide:
1. What data is being presented
2. Extract the key values
3. Any patterns or insights from the data
"""

result = analyze_image(image_path, table_prompt, media_type="image/png")
print(result)

---

## 4. Complex Reasoning Tasks

Mistral Large 3 demonstrates strong reasoning capabilities for multi-step problems.

---

In [ ]:
# Mathematical Reasoning - Probability Problem
math_problem = """
A company has two factories. Factory A produces 60% of the total output, 
while Factory B produces 40%. The defect rate at Factory A is 2%, 
and at Factory B is 3%.

1. What is the probability that a randomly selected product is defective?
2. If a product is found to be defective, what is the probability it came from Factory A?

Show your reasoning step by step.
"""

messages = [{"role": "user", "content": [{"text": math_problem}]}]

system_prompt = "You are a mathematics expert. Solve problems step by step, showing all work clearly."

response = converse(messages, system_prompt=system_prompt, temperature=0.1)
print(get_response_text(response))

In [ ]:
# Business Analysis and Strategic Decision Making
business_analysis = """
You are advising a startup on their cloud infrastructure strategy. Consider the following scenario:

Company Profile:
- B2B SaaS product for data analytics
- Current: 100 enterprise customers, expecting 3x growth in 18 months
- Peak usage: 10 AM - 4 PM EST, weekdays
- Data sensitivity: Financial and PII data
- Current monthly cloud spend: $50,000
- Engineering team: 15 developers, 2 DevOps engineers

They're considering:
A) Moving from multi-cloud to AWS-only to simplify operations
B) Implementing Kubernetes for container orchestration
C) Adopting a serverless-first architecture

Provide a comprehensive analysis with:
1. Pros and cons of each option
2. Risk assessment
3. Your recommended approach with justification
"""

messages = [{"role": "user", "content": [{"text": business_analysis}]}]

system_prompt = "You are a senior cloud architect with expertise in enterprise infrastructure strategy."

response = converse(messages, system_prompt=system_prompt, temperature=0.4, max_tokens=2500)
print(get_response_text(response))

---

## Conclusion

Mistral Large 3 represents a significant advancement in open-source AI models, combining:

- **Frontier Performance**: Ranks among the top open-source models
- **Multimodal Capabilities**: Native vision understanding for image analysis
- **Multilingual Excellence**: 40+ language support with best-in-class performance
- **Advanced Reasoning**: Strong mathematical, logical, and analytical capabilities
- **Open Accessibility**: Apache 2.0 license for flexible deployment

The combination of these capabilities with Amazon Bedrock's managed infrastructure makes Mistral Large 3 an excellent choice for enterprise applications requiring powerful, versatile, and cost-effective AI.

---

## Clean Up

This notebook uses Amazon Bedrock's serverless inference, so there are no resources to clean up. You are charged based on usage (input and output tokens processed).